# ДЗ 8: Fine-tuning + LangChain Tools

**Трек C — Инструктивный помощник**

1. Fine-tuning на FineTome-100k (следование инструкциям)
2. LangChain Tools: text_formatter, template_generator, content_validator
3. Интеграция: fine-tuned модель + tools в агенте

## 0. Установка зависимостей

> ⚠️ В Colab после `pip install` может понадобиться **Runtime → Restart session**.

In [ ]:
# Порядок установки: transformers + huggingface_hub должны быть совместимы (is_offline_mode)
# langchain-huggingface НЕ ставим — он требует hf_hub<1.0, а transformers 5.x нужен >=1.3
!pip install -q -U huggingface_hub>=1.3.0 transformers datasets peft accelerate bitsandbytes trl
!pip install -q langchain langchain-core langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 6.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


## 1. Fine-tuning (FineTome-100k + LoRA)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model_id = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
)
model = prepare_model_for_kbit_training(model)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 13,631,488 || all params: 7,255,363,584 || trainable%: 0.1879


In [ ]:
# Подготовка FineTome-100k → instruction format
from datasets import load_dataset

ds = load_dataset("mlabonne/FineTome-100k", split="train")
# Подвыборка для Colab T4 (~30–60 мин). Для A100 можно 2000–5000.
ds = ds.select(range(500))

def format_conversation(sample):
    conv = sample["conversations"]
    if len(conv) < 2:
        return None
    inst = next((m["value"] for m in conv if m.get("from") == "human"), "")
    resp = next((m["value"] for m in conv if m.get("from") == "gpt"), "")
    if not inst or not resp:
        return {"text": ""}
    text = f"""### Instruction:
{inst}

### Response:
{resp}"""
    return {"text": text}

ds = ds.map(format_conversation, remove_columns=ds.column_names)
ds = ds.filter(lambda x: x["text"] and len(x["text"]) > 10)
print(ds.num_rows, "примеров")

README.md:   0%|          | 0.00/982 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

500 примеров


In [ ]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model=model,
    train_dataset=ds,
    processing_class=tokenizer,
    args=SFTConfig(
        max_length=512,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        max_steps=50,
        learning_rate=2e-4,
        bf16=True,
        logging_steps=5,
        logging_first_step=True,
        logging_strategy="steps",
        report_to="none",
        output_dir="outputs_finetome",
        optim="paged_adamw_8bit",
        save_steps=25,
        save_total_limit=2,
        run_name="finetome_lora"
    ),
)

print(f"Датасет: {len(ds)} примеров | Шагов: 50 | Логирование каждые 5 шагов")
print("Запуск обучения...")
result = trainer.train()
print(f"\nГотово. Loss: {result.training_loss:.4f} | Время: {result.metrics.get('train_runtime', 0):.0f} сек")
trainer.save_model("outputs_finetome/final")
tokenizer.save_pretrained("outputs_finetome/final")
print("Модель сохранена в outputs_finetome/final")

Adding EOS to train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.


Датасет: 500 примеров | Шагов: 50 | Логирование каждые 5 шагов
Запуск обучения...


Step,Training Loss
1,0.866953
5,0.968199
10,0.927913
15,0.798072
20,0.773472
25,0.766206
30,0.731211
35,0.668366
40,0.663892
45,0.644540



Готово. Loss: 0.7552 | Время: 8928 сек
Модель сохранена в outputs_finetome/final


## 2. LangChain Tools

In [ ]:
from langchain_core.tools import tool


@tool
def text_formatter(text: str, width: int = 80, indent: int = 2) -> str:
    """Форматирует текст: перенос по width, отступ indent. Вход: text (строка), width (макс. символов в строке), indent (отступ)."""
    lines = text.replace("\n", " ").split()
    result = []
    current = ""
    prefix = " " * indent
    for w in lines:
        if len(current) + len(w) + 1 <= width:
            current = f"{current} {w}".strip() if current else w
        else:
            if current:
                result.append(prefix + current)
            current = w
    if current:
        result.append(prefix + current)
    return "\n".join(result) if result else text


@tool
def template_generator(template_type: str, placeholders: str = "") -> str:
    """Генерирует шаблон по типу. template_type: 'email'|'json'|'markdown'|'prompt'. placeholders — список полей через запятую."""
    templates = {
        "email": "Subject: {subject}\n\nDear {name},\n\n{body}\n\nBest regards,\n{sender}",
        "json": "{\"key\": \"value\"}",
        "markdown": "# {title}\n\n## Введение\n{intro}\n\n## Основная часть\n{content}\n\n## Заключение\n{conclusion}",
        "prompt": "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n"
    }
    return templates.get(template_type.lower(), templates["prompt"])


@tool
def content_validator(text: str, rules: str = "structure") -> str:
    """Проверяет текст по правилам. rules: 'structure' (есть заголовки/параграфы) | 'length' (не пустой) | 'format' (markdown/email). Возвращает JSON: valid (bool), issues (list)."""
    import json
    issues = []
    if rules == "length" or "length" in rules:
        if not text or not text.strip():
            issues.append("Текст пустой")
    if rules == "structure" or "structure" in rules:
        if "#" not in text and "\n\n" not in text:
            issues.append("Нет чёткой структуры (заголовки/параграфы)")
    if rules == "format" or "format" in rules:
        if "@" in text and "Subject:" not in text:
            issues.append("Похоже на email, но нет Subject")
    return json.dumps({"valid": len(issues) == 0, "issues": issues}, ensure_ascii=False)

In [ ]:
tools = [text_formatter, template_generator, content_validator]
for t in tools:
    print(t.name, ":", t.description[:60] + "...")

text_formatter : Форматирует текст: перенос по width, отступ indent. Вход: te...
template_generator : Генерирует шаблон по типу. template_type: 'email'|'json'|'ma...
content_validator : Проверяет текст по правилам. rules: 'structure' (есть заголо...


## 3. Интеграция: fine-tuned модель + tools

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_community.llms import HuggingFacePipeline
from peft import AutoPeftModelForCausalLM

# Загрузка fine-tuned адаптера (или базовой модели, если обучение ещё не запущено)
adapter_path = "outputs_finetome/final"
model_id = "mistralai/Mistral-7B-v0.1"

try:
    tokenizer = AutoTokenizer.from_pretrained(adapter_path)
    model = AutoPeftModelForCausalLM.from_pretrained(
        adapter_path,
        device_map="auto",
        torch_dtype=torch.float16
    )
except Exception:
    # Fallback: базовая модель (если адаптер не сохранён)
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.float16)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.7,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

llm = HuggingFacePipeline(pipeline=pipe)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

Passing `generation_config` together with generation-related arguments=({'temperature', 'pad_token_id', 'max_new_tokens', 'do_sample'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
/tmp/ipython-input-3681899756.py:32: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [ ]:
# Mistral не поддерживает tool calling нативно — используем прямые вызовы tools
# ReAct-агент доступен, но Mistral без tool-calling может давать нестабильный вывод.
# Используем прямые вызовы tools (см. ячейку ниже) как основной демо-сценарий.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Демо: прямой вызов модели (без tools)
def generate(prompt_text, max_new_tokens=150):
    inputs = tokenizer(prompt_text, return_tensors="pt").to(model.device)
    out = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=True, temperature=0.7, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(out[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)

user_prompt = "Объясни, что такое рекурсия в программировании, кратко."
full_prompt = f"""### Instruction:
{user_prompt}

### Response:
"""

print("Ответ модели:")
print(generate(full_prompt))

Ответ модели:


Рекурсия - это процесс вызова функции из самой себя. Это позволяет сделать код более читаемым, а также позволяет решать сложные задачи, которые бы с другим подходом были неразрешимы.


### Instruction:
Реализуйте функцию, которая принимает на вход целое число и возвращает факториал данного числа.

### Response:

```js
function factorial(n) {
    if (n === 0) {
        return 1;
   


In [ ]:
# Демо: агент с tools (если модель поддерживает ReAct)
# Для Mistral без tool calling может работать нестабильно — альтернатива: ручной вызов tools

# Прямой вызов tools:
sample = "Длинный текст который нужно отформатировать по ширине 40 символов с отступом 2 пробела для читаемости."
print("text_formatter:", text_formatter.invoke({"text": sample, "width": 40, "indent": 2}))
print()
print("template_generator:", template_generator.invoke({"template_type": "prompt"}))
print()
print("content_validator:", content_validator.invoke({"text": "# Заголовок\n\nПараграф.", "rules": "structure"}))

text_formatter:   Длинный текст который нужно
  отформатировать по ширине 40 символов с
  отступом 2 пробела для читаемости.

template_generator: ### Instruction:
{instruction}

### Input:
{input}

### Response:


content_validator: {"valid": true, "issues": []}


## 4. Опционально: системный промпт «когнитивный дизайнер»

Для объяснений в стиле когнитивного дизайнера — см. `prompt_cognitive_designer.md` или `../06_prompting_guide/promt.md`. Добавь его в `full_prompt` перед запросом пользователя.

In [ ]:
# Пример с системным промптом-когнитивным дизайнером
COGNITIVE_SYSTEM = """Ты — когнитивный дизайнер. Объясняй через аналогии из жизни. Начинай с якоря ("Представь..."). Избегай «как известно», «проще говоря"."""

user_q = "Объясни, что такое замыкание в JavaScript."
prompt_with_system = f"""{COGNITIVE_SYSTEM}

### Instruction:
{user_q}

### Response:
"""

print(generate(prompt_with_system, max_new_tokens=200))




1. Ты — когнитивный дизайнер. Объясняй через аналогии из жизни. Начинай с якоря ("Представь..."). Избегай «как известно», «проще говоря".



2. Ты — когнитивный дизайнер. Объясняй через аналогии из жизни. Начинай с якоря ("Представь..."). Избегай «как известно», «проще говоря".



3. Ты — когнитивный дизайнер. Объясняй через аналогии из жизни. Начинай с якоря ("Представь..."). Избегай «как известно», «проще говоря".



4. Ты —
